<a href="https://colab.research.google.com/github/peterbmob/Labs/blob/main/atomorbitaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installera nödvändiga paket till tdin pythonmiljö
%%capture
!pip install ipyvolume 

# Atomorbitaler

Noter: 

- sannolikhetstätheten som funktion av avstånd från kärnan följer $r^2R(r)^2$ där R är den radiella delen av vågfunktionen
- vi använder oss av den reella delen av lösningen till klotytfuntkionsproblemet (eng: "spherical harmonics") 
- atomära enheter används 

Radiella delen av vågfuntionen i atomära enheter kan uttryckas genom ett Laguerreplynom (https://en.wikipedia.org/wiki/Laguerre_polynomials?msclkid=19870a90a9df11ec8ff3feef8ed08262) 


$R_{nl}(r) = \sqrt{\Big(\frac{2}{n a_0}\Big)^3 \frac{(n-l-1)!}{2n (n+l)!}} e^{-r/n a_0} \Big( \frac{2r}{na_0}\Big)^l  \cdot L^{2l+1}_{n-l-1} \Big(\frac{2r}{n a_0} \Big)$

I nedanstående cell defineras denna genom psi_R. 

För att plotta vågfunktionens utbredning i rummet och eventuella vinkelnoder behöver vi även den vinkelberodende delen 

$Y_{lm}(\theta,\phi) = \Theta_{lm}(\theta) \Phi_m (\phi) = \sqrt{\frac{2l+1}{4\pi} \frac{(l-m)!}{(l+m)!} } P_{lm}(cos \theta) \cdot e^{im\phi}$

den defineras i cellen nedan genom ps_ang. 

Tillsammans fås den totala vågfunktionen som: 

$\Psi(r,\theta,\phi) =  R_{nl}(r)Y_{lm}(\theta,\phi)$

### steg 1: ladda in relevanta paket som behövs samt definera hur vi räknar ut $R_{nl}(r)$ och $Y_{lm}(\theta,\phi)$

In [ ]:

import matplotlib.pyplot as plt

from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import scipy.integrate as integrate

# Increase resolution for retina display
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

# Load interactive widgets
import ipywidgets as widgets

import ipyvolume as ipv

# Import special functions 
import scipy.special as spe

import warnings
warnings.filterwarnings('ignore')

def psi_R(r,n=1,l=0):

    coeff = np.sqrt((2.0/n)**3 * spe.factorial(n-l-1) /(2.0*n*spe.factorial(n+l)))
    
    laguerre = spe.assoc_laguerre(2.0*r/n,n-l-1,2*l+1)
    
    return coeff * np.exp(-r/n) * (2.0*r/n)**l * laguerre


def psi_ang(phi,theta,l=0,m=0):
    
    sphHarm = spe.sph_harm(m,l,phi,theta)
    
    return sphHarm.real

### Steg 2: välj kvanttal och bikvvantal för att plotta $R_{nl}(r)$, $R_{nl}(r)^2$ samt $r^2R_{nl}(r)^2$ 

In [ ]:
# välj huvudkvanttal 
n = 4  
# bikvanttal 
l=1

r = np.linspace(0,30,1000) # skapa en vektpr med r-värden
R = psi_R(r,n,l) # kalla på psi_R definerad ovan och fyll på R-vektorn 


# plotta data för de olika fallen
%matplotlib inline
plt.subplots(figsize=(10, 10)) # initiera 

ax1=plt.subplot(311)
plt.plot(r, R, lw=3, axes=ax1)

plt.xlabel('$r [a_0]$',fontsize=20)

plt.ylabel('$R_{nl}(r)$', fontsize=20)

plt.grid('True')

ax2=plt.subplot(312, sharex=ax1)

plt.plot(r, R**2, lw=3, axes=ax2)

plt.xlabel('$r [a_0]$',fontsize=20)

plt.ylabel('$R_{nl}(r)^2$', fontsize=20)

plt.grid('True')

ax3=plt.subplot(313,sharex=ax1)

plt.plot(r, r**2*R**2, lw=3, axes=ax3)

plt.xlabel('$r [a_0]$',fontsize=20)

plt.ylabel('$r^2R_{nl}(r)^2$', fontsize=20)
plt.subplots_adjust(hspace=0.3)
plt.grid('True')

#### Det vi gjorde ovan går även att göra med en sk. widget som tillåter att vi istället ger huvudkvantalet och bikvantalet genom en my i figuren. 

In [ ]:
nmax=10


@widgets.interact(n = np.arange(1,nmax,1), l = np.arange(0,nmax-1,1))
def plot_radial(n=1,l=0):
    
    r =    np.linspace(0,250,10000)
    
    psi2 = psi_R(r,n,l)**2 * (r**2)
    
    plt.plot(r, psi2, lw=2, color='red')
    

    ''' Styling the plot'''
    
    plt.xlabel('$r [a_0]$', fontsize=20)

    plt.ylabel('$r^2R_{nl}(r)^2$',fontsize=20)
    
    rmax = n**2*(1+0.5*(1-l*(l+1)/n**2))
    
    plt.xlim([0, 2*rmax])

#### Steg 3: Så låt oss titta mer på den vinkelberoende delen. Här måste vi ange bikvantalen $l$ och $m_l$ 

In [ ]:
#Ange bikvanttal l
l= 2
# ange magnetiska kvanttalet m
m= 1

phi, theta = np.linspace(0, np.pi, 100), np.linspace(0, 2*np.pi, 100) # skapa vektorer med phi och thetavärden
 
phi, theta = np.meshgrid(phi, theta)

Ylm = psi_ang(theta,phi,l,m)  # beräkna Ylm med funktionen psi_ang vi definerat ovan

#### För att visualisera hur den vinkelberoende delen ser ut i 3D är det enklast att vi går tillbaks till det kartesiska koordinatsystemet. Se här  https://mathworld.wolfram.com/SphericalCoordinates.html?msclkid=207adef1a9e311ec84fd6885b57328a7 för förklaring

In [ ]:
x = np.sin(phi) * np.cos(theta) * abs(Ylm)
y = np.sin(phi) * np.sin(theta) * abs(Ylm)
z = np.cos(phi) * abs(Ylm)

#### och sedan plotta den... 

In [ ]:
%matplotlib inline
'''Set up the 3D Canvas'''

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

''' Normalize color bar to [0,1] scale'''

fcolors = (Ylm - Ylm.min())/(Ylm.max() - Ylm.min())

'''Make 3D plot of real part of spherical harmonic'''

ax.plot_surface(x, y, z, facecolors=cm.seismic(fcolors), alpha=0.3)


''' Project 3D plot onto 2D planes'''

cset = ax.contour(x, y, z,20, zdir='z',offset = -1, cmap='summer')
cset = ax.contour(x, y, z,20, zdir='y',offset =  1, cmap='winter' )
cset = ax.contour(x, y, z,20, zdir='x',offset = -1, cmap='autumn')


''' Set axes limit to keep aspect ratio 1:1:1 '''

ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)